# 🎓 AI4Industry - Module 3 & 4 : RAG avec API Mistral
## Mensaflow © 2025 - Formation CNAM

**Version : API Mistral AI (français 🇫🇷)**

**Objectifs :**
- ✅ RAG simple avec API Mistral
- ✅ Sans installation lourde (pas d'Ollama)
- ✅ Interface Gradio pour import de documents
- ✅ Test avec fake datas GreenPower Solutions

**Stack :** Mistral API + Qdrant + LangChain + Gradio

---

## 📦 Installation (Simple & Rapide)

In [1]:
# Installation des packages Python
!pip install -q langchain==0.3.20 langchain-community==0.3.20 langchain-core==0.3.40
!pip install -q langchain-mistralai  # Package officiel Mistral
!pip install -q qdrant-client==1.7.0
!pip install -q sentence-transformers
!pip install -q gradio
!pip install -q pypdf
!pip install -q python-docx

print("✅ Installation terminée !")
print("📝 Pensez à configurer votre clé API Mistral dans la cellule suivante.")

ERROR: Cannot install langchain-core==0.3.40 and langchain==0.3.20 because these package versions have conflicting dependencies.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A ne

✅ Installation terminée !
📝 Pensez à configurer votre clé API Mistral dans la cellule suivante.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 🔑 Configuration de la clé API Mistral

**Obtenir votre clé API gratuite :**
1. Allez sur : https://console.mistral.ai/
2. Créez un compte (gratuit)
3. Allez dans "API Keys"
4. Créez une nouvelle clé
5. Copiez-collez la clé ci-dessous

**Note :** Le tier gratuit offre des crédits suffisants pour la formation !

In [2]:
import os
from getpass import getpass

# Demander la clé API de manière sécurisée
MISTRAL_API_KEY = getpass("4zu64Ty0fCvBD6olTr9G4qlUG3ZB8CgC")
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY

print("✅ Clé API configurée !")

✅ Clé API configurée !


## 🧠 Initialisation du système RAG

In [3]:
from langchain_mistralai import ChatMistralAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_community.vectorstores import Qdrant

print("🚀 Initialisation du système RAG...")

# 1. Initialiser le LLM Mistral
print("🤖 Connexion à Mistral AI...")
llm = ChatMistralAI(
    model="mistral-small-latest",  # Modèle optimal pour RAG
    temperature=0.7,
    api_key=os.environ["MISTRAL_API_KEY"]
)

# Test rapide
test_response = llm.invoke("Dis bonjour en une phrase")
print(f"✅ Mistral connecté : {test_response.content[:100]}")

# 2. Initialiser les embeddings
print("\n📊 Chargement du modèle d'embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

# 3. Initialiser Qdrant
print("🗄️ Initialisation de Qdrant...")
qdrant_client = QdrantClient(location=":memory:")
collection_name = "greenpower_docs"

# Créer la collection
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# 4. Créer le vectorstore
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=embeddings
)

print("\n✅ Système RAG prêt !")
print("📊 Stack : Mistral AI + Qdrant + LangChain")

Retrying langchain_mistralai.chat_models.ChatMistralAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised LocalProtocolError: Illegal header value b'Bearer '.


🚀 Initialisation du système RAG...
🤖 Connexion à Mistral AI...


Retrying langchain_mistralai.chat_models.ChatMistralAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised LocalProtocolError: Illegal header value b'Bearer '.
Retrying langchain_mistralai.chat_models.ChatMistralAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised LocalProtocolError: Illegal header value b'Bearer '.
Retrying langchain_mistralai.chat_models.ChatMistralAI.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised LocalProtocolError: Illegal header value b'Bearer '.


LocalProtocolError: Illegal header value b'Bearer '

## 📄 Fonctions de traitement de documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from PyPDF2 import PdfReader
import docx
import io

# Text splitter pour chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ". ", " ", ""]
)

def process_pdf(file):
    """Extraire texte d'un PDF"""
    try:
        pdf_reader = PdfReader(io.BytesIO(file))
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Erreur PDF: {str(e)}"

def process_docx(file):
    """Extraire texte d'un DOCX"""
    try:
        doc = docx.Document(io.BytesIO(file))
        text = "\n".join([para.text for para in doc.paragraphs])
        return text
    except Exception as e:
        return f"Erreur DOCX: {str(e)}"

def process_txt(file):
    """Extraire texte d'un TXT"""
    try:
        text = file.decode('utf-8')
        return text
    except Exception as e:
        return f"Erreur TXT: {str(e)}"

def add_documents_to_vectorstore(files, progress=None):
    """Ajouter des documents au vectorstore"""
    if not files:
        return "⚠️ Aucun fichier sélectionné"
    
    all_documents = []
    status_messages = []
    
    for i, file in enumerate(files):
        try:
            filename = file.name if hasattr(file, 'name') else f"file_{i}"
            
            # Lire le contenu du fichier
            file_content = file.read() if hasattr(file, 'read') else file
            
            # Extraire le texte selon le type
            if filename.endswith('.pdf'):
                text = process_pdf(file_content)
            elif filename.endswith('.docx'):
                text = process_docx(file_content)
            elif filename.endswith('.txt'):
                text = process_txt(file_content)
            else:
                status_messages.append(f"⚠️ {filename}: Format non supporté")
                continue
            
            if text.startswith("Erreur"):
                status_messages.append(f"❌ {filename}: {text}")
                continue
            
            # Créer un document
            doc = Document(
                page_content=text,
                metadata={"source": filename}
            )
            
            # Découper en chunks
            chunks = text_splitter.split_documents([doc])
            all_documents.extend(chunks)
            
            status_messages.append(f"✅ {filename}: {len(chunks)} chunks créés")
            
            if progress:
                progress((i + 1) / len(files))
                
        except Exception as e:
            status_messages.append(f"❌ {filename}: {str(e)}")
    
    # Ajouter tous les documents au vectorstore
    if all_documents:
        try:
            vectorstore.add_documents(all_documents)
            status_messages.append(f"\n🎉 Total: {len(all_documents)} chunks indexés dans Qdrant")
        except Exception as e:
            status_messages.append(f"\n❌ Erreur d'indexation: {str(e)}")
    
    return "\n".join(status_messages)

print("✅ Fonctions de traitement de documents prêtes")

## 💬 Fonction de Chat avec RAG

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

def chat_with_rag(message, history):
    """Fonction de chat avec contexte RAG"""
    if not message.strip():
        return "⚠️ Veuillez entrer une question"
    
    try:
        # 1. Rechercher les documents pertinents
        relevant_docs = vectorstore.similarity_search(message, k=3)
        
        if not relevant_docs:
            return "⚠️ Aucun document n'a été indexé. Veuillez d'abord importer des documents."
        
        # 2. Construire le contexte
        context = "\n\n".join([f"Document {i+1} ({doc.metadata.get('source', 'N/A')}):\n{doc.page_content}" 
                                for i, doc in enumerate(relevant_docs)])
        
        # 3. Construire les messages pour Mistral
        system_message = SystemMessage(content="""Tu es un assistant IA spécialisé dans les produits GreenPower Solutions.

Instructions:
- Réponds uniquement en te basant sur le contexte fourni
- Si l'information n'est pas dans le contexte, dis-le clairement
- Sois précis et concis
- Cite la source si pertinent
- Réponds en français""")
        
        human_message = HumanMessage(content=f"""Contexte des documents:
{context}

Question: {message}

Réponse:""")
        
        # 4. Obtenir la réponse du LLM
        messages = [system_message, human_message]
        response = llm.invoke(messages)
        
        return response.content
        
    except Exception as e:
        return f"❌ Erreur: {str(e)}"

print("✅ Fonction de chat prête")

## 🎨 Interface Gradio

In [ ]:
import gradio as gr

# CSS personnalisé
custom_css = """
.gradio-container {
    font-family: 'Arial', sans-serif;
}
.header {
    text-align: center;
    padding: 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 20px;
}
"""

# Créer l'interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    
    # Header
    gr.HTML("""
    <div class="header">
        <h1>🎓 AI4Industry - RAG Vectoriel</h1>
        <h3>Mensaflow © 2025 - Formation CNAM</h3>
        <p>Module 3 & 4 : RAG avec Mistral AI 🇫🇷 + GreenPower Solutions</p>
    </div>
    """)
    
    with gr.Row():
        # Colonne gauche : Import de documents
        with gr.Column(scale=1):
            gr.Markdown("## 📂 Import de Documents")
            gr.Markdown("""
            **Formats supportés:**
            - 📄 PDF (.pdf)
            - 📝 Texte (.txt)
            - 📃 Word (.docx)
            
            **Astuce:** Importez les fichiers GreenPower Solutions pour tester le RAG !
            """)
            
            file_input = gr.File(
                label="Sélectionnez vos documents",
                file_count="multiple",
                file_types=[".pdf", ".txt", ".docx"]
            )
            
            upload_btn = gr.Button("📤 Indexer les documents", variant="primary")
            upload_status = gr.Textbox(
                label="Status d'indexation",
                lines=10,
                interactive=False
            )
            
            gr.Markdown("""
            ---
            ### 📊 Statistiques
            """)
            
            stats_btn = gr.Button("🔍 Afficher statistiques")
            stats_output = gr.Textbox(
                label="Statistiques Qdrant",
                lines=6,
                interactive=False
            )
        
        # Colonne droite : Chat
        with gr.Column(scale=2):
            gr.Markdown("## 💬 Chat avec RAG")
            gr.Markdown("""
            Posez vos questions sur les documents GreenPower Solutions indexés.
            Le système utilisera **Mistral AI** 🇫🇷 pour générer des réponses basées sur le contexte.
            """)
            
            chatbot = gr.Chatbot(
                height=500,
                label="Assistant GreenPower (Mistral AI)",
                avatar_images=(None, "🇫🇷")
            )
            
            msg = gr.Textbox(
                label="Votre question",
                placeholder="Ex: Quels sont les produits GreenPower disponibles ?",
                lines=2
            )
            
            with gr.Row():
                submit_btn = gr.Button("🚀 Envoyer", variant="primary")
                clear_btn = gr.Button("🗑️ Effacer", variant="secondary")
            
            gr.Markdown("""
            ---
            ### 💡 Questions suggérées:
            - Quels sont les produits disponibles ?
            - Quelle est la capacité batterie du PG-M01 ?
            - Quel système convient pour un festival de 3 jours ?
            - Combien coûte le PG-P01 ?
            - Quels événements ont été organisés en 2024 ?
            """)
    
    # Footer
    gr.HTML("""
    <div style="text-align: center; padding: 20px; color: #666;">
        <p><strong>Stack Technique:</strong> Mistral AI 🇫🇷 + Qdrant + LangChain + Gradio</p>
        <p><strong>Modèle:</strong> mistral-small-latest (optimisé pour RAG)</p>
        <p>Développé par Mensaflow pour AI4Industry CNAM 2025</p>
    </div>
    """)
    
    # Événements
    def get_stats():
        try:
            collection_info = qdrant_client.get_collection(collection_name)
            return f"""📊 Statistiques Qdrant:
            
📦 Collection: {collection_name}
📝 Nombre de vectors: {collection_info.vectors_count}
📏 Dimension: 384
📐 Distance: Cosine
🤖 LLM: Mistral AI (mistral-small-latest)
✅ Status: Opérationnel
"""
        except Exception as e:
            return f"❌ Erreur: {str(e)}"
    
    def respond(message, chat_history):
        bot_message = chat_with_rag(message, chat_history)
        chat_history.append((message, bot_message))
        return "", chat_history
    
    # Lier les événements
    upload_btn.click(
        fn=add_documents_to_vectorstore,
        inputs=[file_input],
        outputs=[upload_status]
    )
    
    stats_btn.click(
        fn=get_stats,
        outputs=[stats_output]
    )
    
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(lambda: None, None, chatbot, queue=False)

# Lancer l'interface
print("\n🚀 Lancement de l'interface Gradio...")
print("📱 Un lien public sera généré pour partager avec les étudiants\n")
demo.launch(share=True, debug=True)

---
## 📚 Guide d'utilisation pour les étudiants

### ✅ Avantages de cette version (API Mistral)

**Pas de galère technique :**
- ✅ Pas d'installation lourde (pas d'Ollama)
- ✅ Pas de téléchargement de modèle (4GB économisés)
- ✅ Pas de redémarrage nécessaire
- ✅ Prêt en 2 minutes

**Performance :**
- ✅ Mistral AI = modèle français de qualité
- ✅ Inference rapide (API cloud)
- ✅ Pas de limite GPU/CPU

**Pédagogique :**
- ✅ Focus sur le RAG, pas sur l'infrastructure
- ✅ Argument "souveraineté française"
- ✅ Expérience identique pour tous les étudiants

---

### 🎯 Étapes pour utiliser ce notebook

**1. Obtenir une clé API Mistral (gratuit)**
- Allez sur https://console.mistral.ai/
- Créez un compte
- Générez une clé API
- Collez-la dans la cellule "Configuration"

**2. Exécuter les cellules**
- Exécutez toutes les cellules dans l'ordre
- Temps total : ~2 minutes
- Pas de redémarrage nécessaire !

**3. Importer les documents GreenPower**
- Cliquez sur "Sélectionnez vos documents"
- Uploadez les PDF/TXT GreenPower
- Cliquez "Indexer les documents"

**4. Tester le RAG**
- Posez des questions dans le chat
- Observez comment Mistral répond avec le contexte

---

### 📊 Questions d'analyse pour la formation

**Comprendre le RAG :**
1. Comment le système trouve-t-il les documents pertinents ?
2. Qu'est-ce qu'un "chunk" et pourquoi c'est important ?
3. Que se passe-t-il si on pose une question hors contexte ?

**Limites du RAG vectoriel :**
1. Le RAG peut-il répondre à "Quel produit a la meilleure autonomie ET le prix le plus bas" ?
2. Peut-il faire des comparaisons multi-critères ?
3. Peut-il suivre des relations complexes ("Si X alors Y") ?

**→ Ces limites justifient le passage au GraphRAG (Module 5) !**

---

### 🎓 Objectifs pédagogiques atteints

✅ Architecture RAG complète  
✅ Chunking et embeddings  
✅ Vector database (Qdrant)  
✅ LLM API (Mistral AI français)  
✅ Interface utilisateur (Gradio)  
✅ Prêt pour Module 5 (GraphRAG)  

---

## 🔄 Comparaison des versions

| Critère | API Mistral (cette version) | Ollama Local |
|---------|----------------------------|---------------|
| Installation | ⚡ Simple (2 min) | ⏳ Complexe (10 min) |
| Redémarrage | ✅ Non | ❌ Oui (obligatoire) |
| Téléchargement | ✅ Rien | ❌ 4GB (Mistral 7B) |
| Performance | 🚀 Rapide (cloud) | 🐌 Variable (CPU) |
| Coût | 💰 Gratuit (tier) | 💻 Local (gratuit) |
| Souveraineté | 🇫🇷 Français | 🌍 Local |
| Pédagogie | ✅ Focus RAG | ⚠️ Focus infra |

**Recommandation :** API Mistral pour la formation, Ollama pour travail hors ligne

---

**Mensaflow © 2025 - Formation AI4Industry CNAM**